# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
# Import API key
from config import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('output_data/cities.csv')
cities_df.head()


,city,lat,lng,max temperature,humidity,cloudiness,wind speed,country,date
0,dhaka,23.786198,90.402615,82.38,78.0,40.0,4.61,BD,1.659133e+09
1,kodiak,57.790000,-152.407222,56.71,88.0,100.0,10.36,US,1.659133e+09
2,dikson,73.508389,80.512382,40.98,96.0,100.0,16.42,RU,1.659133e+09
3,luwuk,-0.952823,122.788020,75.42,94.0,99.0,3.98,ID,1.659133e+09
4,port alfred,-33.598184,26.891477,59.22,79.0,15.0,12.71,ZA,1.659133e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = cities_df[['lat', 'lng']]
humidity = cities_df['humidity'].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Max Temp lower than 80 but higher than 70
# Wind speed less than 10 mph
# Cloudiness less than or equal to 50
# using .loc
ideal_weather_df = cities_df.loc[(cities_df['max temperature'] < 80) & (cities_df['max temperature'] >= 70) & (cities_df['wind speed'] < 10) & (cities_df['cloudiness'] <= 50)]
ideal_weather_df

,city,lat,lng,max temperature,humidity,cloudiness,wind speed,country,date
72,besikduzu,41.052731,39.228038,73.49,85.0,0.0,3.83,TR,1.659134e+09
88,ponta do sol,32.681067,-17.104073,74.70,82.0,2.0,4.61,PT,1.659134e+09
118,ayios matthaios,39.493969,19.872864,75.38,47.0,0.0,3.44,GR,1.659134e+09
120,benicarlo,40.418024,0.422981,78.89,82.0,25.0,1.90,ES,1.659134e+09
126,saint-pierre,48.383272,7.471873,71.69,61.0,50.0,4.12,FR,1.659134e+09
127,ribeira grande,37.822523,-25.519491,71.94,83.0,20.0,9.22,PT,1.659134e+09
128,belmonte,39.559283,-2.703812,74.53,29.0,0.0,7.67,ES,1.659134e+09
129,muros,40.678061,8.617315,73.85,74.0,0.0,1.95,IT,1.659134e+09
145,babu,24.415543,111.546466,76.66,84.0,26.0,2.01,CN,1.659134e+09
148,nogent-le-rotrou,48.321809,0.821149,70.52,47.0,43.0,2.30,FR,1.659134e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather_df.copy()
hotel_df['Hotel Name'] = ""
hotel_df = hotel_df.drop(columns=['date'])
hotel_df

,city,lat,lng,max temperature,humidity,cloudiness,wind speed,country,Hotel Name
72,besikduzu,41.052731,39.228038,73.49,85.0,0.0,3.83,TR,
88,ponta do sol,32.681067,-17.104073,74.70,82.0,2.0,4.61,PT,
118,ayios matthaios,39.493969,19.872864,75.38,47.0,0.0,3.44,GR,
120,benicarlo,40.418024,0.422981,78.89,82.0,25.0,1.90,ES,
126,saint-pierre,48.383272,7.471873,71.69,61.0,50.0,4.12,FR,
127,ribeira grande,37.822523,-25.519491,71.94,83.0,20.0,9.22,PT,
128,belmonte,39.559283,-2.703812,74.53,29.0,0.0,7.67,ES,
129,muros,40.678061,8.617315,73.85,74.0,0.0,1.95,IT,
145,babu,24.415543,111.546466,76.66,84.0,26.0,2.01,CN,
148,nogent-le-rotrou,48.321809,0.821149,70.52,47.0,43.0,2.30,FR,


In [7]:
# Loop through Hotel Dataframe
# Using google places API nearbysearch
# Setting up base url and parameters
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
parameters = {
    'location': "",
    'radius': 5000,
    'type': 'lodging',
    'key': g_key,
}

for index, row in hotel_df.iterrows():
    # grab location
    lat = row['lat']
    lng = row['lng']
    
    # put location into parameters
    parameters['location'] = f'{lat},{lng}'
    
    # get json response for each location
    print(f'retrieving results for {index}: {lat},{lng}')
    response = requests.get(base_url, params=parameters).json()
    result = response['results']
#     pprint(result)
    try:
        print('hotel found')
        name = result[0]['name']
        print(f'Nearest Hotel: {name}')
        
        hotel_df.loc[index, 'Hotel Name'] = result[0]['name']
        
    except (KeyError, IndexError):
        print('Hotels not found')


retrieving results for 72: 41.0527305,39.2280377
hotel found
Nearest Hotel: Belek Palas Hotel
retrieving results for 88: 32.681067,-17.104073
hotel found
Nearest Hotel: Hotel do Campo
retrieving results for 118: 39.4939685,19.8728635
hotel found
Nearest Hotel: Paramonas Hotel
retrieving results for 120: 40.4180244,0.4229813
hotel found
Nearest Hotel: Hotel & Spa Peñíscola Plaza Suites
retrieving results for 126: 48.3832725,7.4718731
hotel found
Nearest Hotel: Hôtel-Restaurant Les Hortensias
retrieving results for 127: 37.8225227,-25.5194914
hotel found
Nearest Hotel: Casa do Pico Arde
retrieving results for 128: 39.5592831,-2.7038117
hotel found
Nearest Hotel: Palacio Infante Don Juan Manuel
retrieving results for 129: 40.678061,8.617315
hotel found
Nearest Hotel: Mario Home
retrieving results for 145: 24.4155435,111.54646629999999
hotel found
Nearest Hotel: IU Hotel Hezhou Bus Terminal
retrieving results for 148: 48.321809,0.8211488000000001
hotel found
Nearest Hotel: Logis Hôtel Sull

In [8]:
# Droping Rows with no Hotels 
# Replacing blank('') with np.nan
# Drop na
hotel_df
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,city,lat,lng,max temperature,humidity,cloudiness,wind speed,country,Hotel Name
72,besikduzu,41.052731,39.228038,73.49,85.0,0.0,3.83,TR,Belek Palas Hotel
88,ponta do sol,32.681067,-17.104073,74.70,82.0,2.0,4.61,PT,Hotel do Campo
118,ayios matthaios,39.493969,19.872864,75.38,47.0,0.0,3.44,GR,Paramonas Hotel
120,benicarlo,40.418024,0.422981,78.89,82.0,25.0,1.90,ES,Hotel & Spa Peñíscola Plaza Suites
126,saint-pierre,48.383272,7.471873,71.69,61.0,50.0,4.12,FR,Hôtel-Restaurant Les Hortensias
127,ribeira grande,37.822523,-25.519491,71.94,83.0,20.0,9.22,PT,Casa do Pico Arde
128,belmonte,39.559283,-2.703812,74.53,29.0,0.0,7.67,ES,Palacio Infante Don Juan Manuel
129,muros,40.678061,8.617315,73.85,74.0,0.0,1.95,IT,Mario Home
145,babu,24.415543,111.546466,76.66,84.0,26.0,2.01,CN,IU Hotel Hezhou Bus Terminal
148,nogent-le-rotrou,48.321809,0.821149,70.52,47.0,43.0,2.30,FR,Logis Hôtel Sully


In [9]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))